# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import config
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "metadata.csv"), index_col=[0, 1]).iloc[:, [0]]
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283294        0.000000                     0.000000   
         SAMN03283288        0.011865                     0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283294                                            0.000000   
         SAMN03283288                                            0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283294                                               0.000000                      
         SAMN03283288                                               0.011865                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283294                                               0.000000                                             
         SAMN03283288                                               0.011865                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283294                                               0.000000                                                                   
         SAMN03283288                                               0.011865                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283294                              True
         SAMN03283288                              True

# Generate predictions in 10-fold cross validation! (repeated 10 times)

In [3]:
np.random.seed(42)

In [4]:
num_repeats = 10
n_splits = 10

logit_prediction_list = []

for i in tqdm(range(num_repeats)):
    
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=np.random.randint(0, high=1000000))
    
    # instantiate the logistic regression classifier (gmhi2)
    gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")

    # generate logit predictions in 10 fold cross validation
    GMHI2_scores_cv = cross_val_predict(gmhi2, X > config.PRESENCE_CUTOFF, y.values.flatten(), method="decision_function",
        cv=kfold, verbose=0, n_jobs=-1
    )

    GMHI2_scores_cv = pd.DataFrame(GMHI2_scores_cv, index=y.index, columns=["GMHI2_cv"])
    logit_prediction_list.append(GMHI2_scores_cv)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:21<00:00,  2.20s/it]


In [5]:
def evaluate_performance(cutoff):
    balanced_accuracies = [balanced_accuracy_score(y[(abs(lo) >= cutoff).values], lo[(abs(lo) >= cutoff).values] > 0) for lo in logit_prediction_list]
    mean_acc = np.mean(balanced_accuracies)
    print("Mean acc:", mean_acc)
    
    mat_sum = np.zeros([2 , 2])

    for lo in logit_prediction_list:
        idx = (abs(lo) >= cutoff).values
        y_curr = y[idx]
        lo_curr = lo[idx]
        mat = confusion_matrix(y_curr, lo_curr > 0)
        mat_sum += mat
    tn, fp, fn, tp = mat_sum.ravel()
    df = pd.DataFrame(mat_sum, columns=["Predicted Nonhealthy", "Predicted Healthy"], index=["Actual Nonhealthy", "Actual healthy"])
    display(df)
    print("Total samples evaluated:", mat_sum.sum())
    print("Percentage of possible:", mat_sum.sum() / X.shape[0] * 10)

In [6]:
evaluate_performance(0)

Mean acc: 0.7856855662349299


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,20902.0,5668.0
Actual healthy,11943.0,43527.0


Total samples evaluated: 82040.0
Percentage of possible: 100.0


In [7]:
evaluate_performance(0.1)

Mean acc: 0.7982565692911956


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,20243.0,5072.0
Actual healthy,10772.0,42257.0


Total samples evaluated: 78344.0
Percentage of possible: 95.49488054607508


In [8]:
evaluate_performance(1)

Mean acc: 0.8994251127023606


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,12696.0,1413.0
Actual healthy,3288.0,29267.0


Total samples evaluated: 46664.0
Percentage of possible: 56.87957094100439
